In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from crewai import Agent, Task, Crew, Process, LLM

In [2]:
# Load Secrets
success = load_dotenv(find_dotenv())

if success:
    print("Secrets loaded successfully")
    #print(os.getenv("GEMINI_API_KEY"))
    
else:
    print("Failed to load secrets")

Secrets loaded successfully


In [3]:
# Setup the brains

# Manager (Cloud - High Intelligence)
manager_llm = LLM(
    model="gemini-2.5-flash",
    api_key=os.getenv("GEMINI_API_KEY")
)

# Worker (Local - Privay & Free Compute)
worker_llm = LLM(
    model="ollama/llama3.1",
    base_url="http://localhost:11434"
)


In [4]:
# Define Workers (Using Local LLM)
# Note - Manager Agent is not defined, CrewAI will spawn it automatically

market_analyst_agent = Agent(
    role="Market Analyst",
    goal="Analyse Market demand and competitors.",
    backstory="You are a sharp businss analyist. You demand data before believing any hype.",
    verbose=True,
    llm=worker_llm,
    allow_delegation=False
)

tech_analyst_agent = Agent(
    role="CTO",
    goal="Assess techical feasibility.",
    backstory="You are an expect technologist and CTO. you judge if the idea is technologically feasible.",
    verbose=True,
    llm=worker_llm,
    allow_delegation=False
)

In [5]:
# Define the high-level foal. 
# We give the goal to the manager, not the workers.

task_validate = Task(
    description = """
    Validate the startup idea: 'A market for 20000 GBP human-sized and human-looking robot that can be trained to perform household chores, and manual tasks inside a household or indoor industry or businesses'    
    1. Ask the "Market Analyst" to investigate if there is market for this idea.
    2. Ask the "CTO" to check if the idea is technologically feasible.
    3. Synnthesize thier answers into a final Go/No-Go decision.
    """,
    expected_output = "A final executive summary with Go/No-Go decision.",
    # No agent assigned here. The Manager will handle it.
)

In [6]:
# The Heirarchical crew

crew = Crew(
    agents=[market_analyst_agent, tech_analyst_agent], # The workers
    tasks=[task_validate], # high-level goal
    process=Process.hierarchical, # Boss Mode activated
    manager_llm=manager_llm, # Uses cloud API
    verbose=True
)

In [7]:
print("### Starting the Crew ###")
print("-------------------------")

resut = crew.kickoff()
print("\n#################\n")
print(resut)
print("#################\n")


### Starting the Crew ###
-------------------------


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 555b73a6-b442-4af5-b17d-b9bf2440caeb                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Validate the startup idea: 'A market for 20000 GBP human-sized and human-looking robot that can be         │
│  trained to perform household chores, and manual tasks inside a household or indoor industry or businesses'     │
│      1. Ask the "Market Analyst" to investigate if there is market for this idea.                               │
│      2. Ask the "CTO" to check if the idea is technologically feasible.                                         │
│      3. Synnthesize thier answers into a final Go/No-Go decision.                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Executive Summary: Go/No-Go Decision                                                                           │
│                                                                                                                 │
│  **Startup Idea:** A market for 20,000 GBP human-sized and human-looking robot that can be trained to perform   │
│  household chores, and manual tasks inside a household or indoor industry or businesses.                        │
│                                                                                                                 │
│  **Market Analysis (Market Analyst):**                                                                          │
│  The Market Analyst indicates a viable market for such a robot, identifying diverse target customer segments    │
│  including the aging population, busy professionals, families with young children, and people with              │
│  disabilities. Demand indicators (search volume, social media buzz, survey results) show moderate interest.     │
│  Projections suggest a market size of 1 billion GBP in Year 1, growing to 1.65 billion GBP in Year 5, based on  │
│  a 20,000 GBP price point and a conservative 10% household adoption rate. While competitors exist (e.g.,        │
│  iRobot Roomba, Samsung Powerbot, humanoid robots), none offer the same comprehensive functionality at the      │
│  proposed price point. The market analyst's conclusion is that there is a viable market.                        │
│                                                                                                                 │
│  **Technological Feasibility (CTO):**                                                                           │
│  The CTO's assessment highlights significant technological challenges. The development of a human-sized robot   │
│  requires substantial R&D investment, collaboration with experts, and considerable manufacturing                │
│  infrastructure. Crucially, the CTO deems achieving the target price of 20,000 GBP within a reasonable          │
│  timeframe (1-3 years) as unlikely. A more realistic estimate for a human-sized robot with similar              │
│  capabilities is projected to be between 50,000 - 250,000 GBP. While acknowledging that modular design, mass    │
│  production, and targeting specific niches could help, the immediate feasibility at the proposed price is low.  │
│                                                                                                                 │
│  **Synthesis and Decision:**                                                                                    │
│  The Market Analyst's positive outlook on market viability is heavily predicated on the 20,000 GBP price point  │
│  for the robot. The projected market size and demand are based on this assumption. However, the CTO's analysis  │
│  clearly states that the 20,000 GBP price is not technologically feasible within a reasonable timeframe, with   │
│  a more realistic cost being 2.5 to 12.5 times higher.                                                          │
│                                                                                                                 │
│  This significant discrepancy between the market-desired price and the technologically achievable price         │
│  renders the current startup idea unviable as proposed.

Output()


#################


╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f06d8a2d-0c4d-4455-9bd7-8c4520040317                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯